# Machine Learning Water Table Analysis

In [149]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelBinarizer

In [150]:
df = pd.read_csv("wt_for_ml.csv")

In [151]:
df.head()

,amount_tsh,gps_height,longitude,latitude,num_private,population,construction_year,basin_encoded,region_encoded,recorded_by_encoded,...,quantity_encoded,quantity_group_encoded,source_encoded,source_type_encoded,source_class_encoded,waterpoint_type_encoded,waterpoint_type_group_encoded,status_group_encoded,public_meeting_encoded,permit_encoded
0,3.799832,1.041377,-0.081164,-1.408734,NaN,-0.440466,0.175277,1,3,0,...,1,1,8,7,0,1,1,0,1,0
1,NaN,1.054363,-0.172951,1.207935,NaN,0.220285,1.057224,4,9,0,...,2,2,5,4,1,1,1,0,1,1
2,-0.532695,0.025591,0.886294,0.639765,NaN,0.104364,0.977047,6,8,0,...,1,1,0,1,1,2,1,0,1,1
3,NaN,-0.584746,1.279593,-1.849654,NaN,-0.637532,-0.867023,8,12,0,...,0,0,3,0,0,2,1,2,1,1
4,NaN,-0.964223,-1.541321,1.317270,NaN,NaN,NaN,4,4,0,...,3,3,5,4,1,1,1,0,1,1


In [152]:
df.shape

(59399, 30)

In [153]:
df.describe()

,amount_tsh,gps_height,longitude,latitude,num_private,population,construction_year,basin_encoded,region_encoded,recorded_by_encoded,...,quantity_encoded,quantity_group_encoded,source_encoded,source_type_encoded,source_class_encoded,waterpoint_type_encoded,waterpoint_type_group_encoded,status_group_encoded,public_meeting_encoded,permit_encoded
count,7872.000000,59397.000000,57587.000000,59399.000000,5.230000e+02,37289.000000,38690.000000,59399.000000,59399.000000,59399.0,...,59399.000000,59399.000000,59399.000000,59399.000000,59399.000000,59399.000000,59399.000000,59399.000000,59399.000000,59399.000000
mean,0.000117,0.000004,-0.000051,0.000024,4.203139e-17,0.000002,0.000071,4.538241,9.835974,0.0,...,1.325746,1.325746,6.063183,4.758262,0.233741,2.573983,2.056129,0.841125,0.914914,0.705500
std,1.000100,1.000015,0.999998,0.999996,1.000000e+00,1.000026,0.999984,2.896863,5.937108,0.0,...,0.803752,0.803752,2.034203,2.575739,0.434131,1.781399,1.378897,0.949796,0.279011,0.455821
min,-0.550678,-1.094082,-2.125701,-2.017385,-9.827165e-01,-1.634453,-2.951625,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-0.514568,-0.964223,-0.715123,-0.962192,-6.686875e-01,-0.726405,-0.786846,2.000000,4.000000,0.0,...,1.000000,1.000000,5.000000,4.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000
50%,-0.369546,-0.431801,-0.055158,0.232384,-4.593349e-01,-0.282040,0.255454,4.000000,10.000000,0.0,...,1.000000,1.000000,7.000000,6.000000,0.000000,2.000000,1.000000,0.000000,1.000000,1.000000
75%,-0.115757,0.938933,0.799204,0.807839,4.827520e-01,0.297565,0.896870,7.000000,15.000000,0.0,...,2.000000,2.000000,8.000000,7.000000,0.000000,4.000000,3.000000,2.000000,1.000000,1.000000
max,6.700269,2.827660,1.992570,1.936864,2.576278e+00,5.282174,1.297755,9.000000,20.000000,0.0,...,4.000000,4.000000,9.000000,7.000000,2.000000,6.000000,5.000000,2.000000,1.000000,1.000000


In [154]:
df.columns

Index(['amount_tsh', 'gps_height', 'longitude', 'latitude', 'num_private',
       'population', 'construction_year', 'basin_encoded', 'region_encoded',
       'recorded_by_encoded', 'scheme_management_encoded',
       'extraction_type_encoded', 'extraction_type_group_encoded',
       'extraction_type_class_encoded', 'management_encoded',
       'management_group_encoded', 'payment_encoded', 'payment_type_encoded',
       'water_quality_encoded', 'quality_group_encoded', 'quantity_encoded',
       'quantity_group_encoded', 'source_encoded', 'source_type_encoded',
       'source_class_encoded', 'waterpoint_type_encoded',
       'waterpoint_type_group_encoded', 'status_group_encoded',
       'public_meeting_encoded', 'permit_encoded'],
      dtype='object')

In [155]:
df.dtypes

amount_tsh                       float64
gps_height                       float64
longitude                        float64
latitude                         float64
num_private                      float64
population                       float64
construction_year                float64
basin_encoded                      int64
region_encoded                     int64
recorded_by_encoded                int64
scheme_management_encoded          int64
extraction_type_encoded            int64
extraction_type_group_encoded      int64
extraction_type_class_encoded      int64
management_encoded                 int64
management_group_encoded           int64
payment_encoded                    int64
payment_type_encoded               int64
water_quality_encoded              int64
quality_group_encoded              int64
quantity_encoded                   int64
quantity_group_encoded             int64
source_encoded                     int64
source_type_encoded                int64
source_class_enc

### Which Algorithms 
Well the outcome var is multi-class categorical -> classification

The task description doesn't give any implication of the importance of explainability, but it seems like it would
be good to know how much each factor affects outcome

Also speed vs accuracy, not mentioned but I'd guess accuracy is more important, as long as I can run it within the time limit, which actually isn't that long

Features include continuous and categorical

In no particular order, options:
    1. Elastic-net regularized logistic regression
    2. Boosted Trees
    3. NN
    4. Kernel SVM
    
I'm gonna go with the boosted trees first, seems like the best bet for these parameters. Does well without a tonne of training data, has good libraries that easily ingest categorical features. 

### Split into training set


In [156]:
# Is there any risk of circularity here? 
# Construction year was the only var that correlated with outcome var with > 0.1 corr coeff
# If it were categorical I'd split into training set by construction year but it isn't

In [157]:
# Split into training and test. 
# TODO validation?
train, test = train_test_split(df, test_size=0.25, random_state=25)
train_y = train['status_group_encoded']
train_x = train.drop(columns=['status_group_encoded'])
test_y = test['status_group_encoded']
test_x = test.drop(columns=['status_group_encoded'])

In [158]:
cat_feats = [x for x in df.columns if '_encoded' in x]
cat_feats.remove('status_group_encoded')

In [160]:
train_x.shape

(44549, 29)

In [161]:
test_y.shape

(14850,)

## Boosted Trees

In [164]:
## Train and run boosted trees LGB

# setup input in the right format
d_train=lgb.Dataset(train_x, label=train_y)

# establish model
lg = lgb.LGBMClassifier(silent=False)

#training the model
params = {"max_depth": 50, "objective" : ["multiclass"], "num_class" : [3], 
          "learning_rate" : 0.1, "num_leaves": 900,  "n_estimators": 300}
model=lgb.train(params, d_train, 100, categorical_feature = cat_feats)  #training the model on 100 epocs

#prediction on the test dataset
y_pred_1=model.predict(test_x)

#printing the predictions
y_pred_1

/home/ian/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/ian/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['basin_encoded', 'extraction_type_class_encoded', 'extraction_type_encoded', 'extraction_type_group_encoded', 'management_encoded', 'management_group_encoded', 'payment_encoded', 'payment_type_encoded', 'permit_encoded', 'public_meeting_encoded', 'quality_group_encoded', 'quantity_encoded', 'quantity_group_encoded', 'recorded_by_encoded', 'region_encoded', 'scheme_management_encoded', 'source_class_encoded', 'source_encoded', 'source_type_encoded', 'water_quality_encoded', 'waterpoint_type_encoded', 'waterpoint_type_group_encoded']
  'New categorical_feature is {}'.format(sorted(list(categorical_fe

array([[7.94368695e-01, 1.42435159e-01, 6.31961459e-02],
       [3.56242050e-05, 7.56364283e-08, 9.99964300e-01],
       [8.51240445e-01, 7.49631073e-02, 7.37964476e-02],
       ...,
       [1.76680469e-02, 1.71352207e-05, 9.82314818e-01],
       [3.54353882e-04, 1.21798240e-05, 9.99633466e-01],
       [9.99973341e-01, 2.69668576e-06, 2.39620356e-05]])

In [165]:
# We get probabilities for each outcome class
# Take the most likely of these as our outcome
preds = [list(x).index(max(x)) for x in y_pred_1]

In [167]:
def multiclass_roc_auc_score(y_test, y_pred, average="macro"):
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    y_pred = lb.transform(y_pred)
    return roc_auc_score(y_test, y_pred, average=average)

In [168]:
multiclass_roc_auc_score(test_y, preds)

0.7686688100776459